In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()
###
!pip install torch
###
!pip install torchvision
!pip install d2l==1.0.0b0
!conda install -c conda-forge torchmetrics

⏬ Downloading https://github.com/jaimergp/miniforge/releases/latest/download/Mambaforge-colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:30
🔁 Restarting kernel...
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.4/887.4 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 55.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 77.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.2/24.2 MB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 82.7 MB/s eta 0:00:00
     ━━━━━

Solving environment: / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - torchmetrics


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _openmp_mutex-4.5          |       2_kmp_llvm           6 KB  conda-forge
    ca-certificates-2022.12.7  |       ha878542_0         143 KB  conda-forge
    certifi-2022.12.7          |     pyhd8ed1ab_0         147 KB  conda-forge
    colorama-0.4.6             |     pyhd8ed1ab_0          25 KB  conda-forge
    conda-23.1.0               |   py38h578d9bd_0         907 KB  conda-forge
    cudatoolkit-11.8.0         |      h37601d7_11       635.9 MB  conda-forge
    cudnn-8.4.1.50             |   

In [ ]:
!pip install wandb -qU
import wandb
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: amoseley018. Use `wandb login --relogin` to force relogin


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import os
import torch
import torchvision
from torchvision import transforms
from torch import nn, Tensor
from torch.nn import functional as F
from torch.utils.data import TensorDataset, DataLoader, Dataset
from torchsummary import summary
from d2l import torch as d2l
from pathlib import Path
import sys
import nibabel as nib
import numpy as np
import gc
import skimage
import h5py
import math
import PIL
from PIL import Image

/usr/local/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


In [ ]:
class FocalLoss(nn.Module):
    def __init__(self, weight0=1, weight1=1, gamma=0):
        super().__init__()

        self.weight0 = weight0
        self.weight1 = weight1
        self.gamma = gamma

    def forward(self, input, target):
        loss = 0
        for i, el in enumerate(input):
            if target[i] == 1:
                loss += torch.log(el) * (abs(1 - el) ** self.gamma) * self.weight1
            else:
                loss += torch.log(1 - el) * (abs(0 - el) ** self.gamma) * self.weight0

        return -1 * loss / len(input)

In [ ]:
class BalancedCELoss(nn.Module):
    def __init__(self, weight0=1, weight1=1):
        super().__init__()

        self.weight0 = weight0
        self.weight1 = weight1

    def forward(self, input, target):
        loss = 0
        for i, el in enumerate(input):
            if target[i] == 1:
                loss += torch.log(el) * self.weight1
            else:
                loss += torch.log(1 - el) * self.weight0

        return -1 * loss / len(input)

In [ ]:
class LITSBinaryDataset(Dataset):
    def __init__(self, fileName):
        super().__init__()

        self.file = h5py.File(fileName, 'r')

        self.length = len(list(self.file.keys()))

    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        data = self.file["Slice" + str(idx)]["Slice"]
        label = self.file["Slice" + str(idx)].attrs.get("ImageLabel")

        result = []

        result.append(torch.Tensor(data[...]).unsqueeze(0))
        result.append(torch.Tensor(label).squeeze(0))

        return result

    def closeFile(self):
        self.file.close()

In [ ]:
class convBlock(nn.Module):
    def __init__(self, inChannels, outChannels, batchNorm, strides, layerMean, layerDev) -> None:
        super().__init__()

        self.conv1 = nn.Conv2d(inChannels, outChannels, kernel_size=3, padding=1, stride=strides)
        self.conv2 = nn.Conv2d(outChannels, outChannels, kernel_size=3, padding=1)

        nn.init.normal_(self.conv1.weight, mean=layerMean, std=layerDev)
        nn.init.normal_(self.conv2.weight, mean=layerMean, std=layerDev)

        if(batchNorm):
            self.bn1 = nn.BatchNorm2d(outChannels)
        else:
            self.bn1 = False

    def forward(self, X):
        Y = self.conv1(X)

        if(self.bn1):
            Y = self.bn1(Y)

        Y = F.relu(Y)

        return torch.Tensor(F.relu(self.conv2(Y)))

In [ ]:
class EncoderBlock(nn.Module):
    def __init__(self, inChannels, outChannels, strides) -> None:
        super().__init__()

        self.conv = convBlock(inChannels, outChannels, True, strides, 0, 0.025)
        self.pool = nn.MaxPool2d(2, stride=strides)

    def forward(self, X):

        Y = self.conv.forward(X)

        #return torch.Tensor(self.pool(Y)), Y
        return self.pool(Y)

In [ ]:
class DecoderBlock(nn.Module):
    def __init__(self, inChannels, outChannels, strides) -> None:
        super().__init__()

        self.convTrans = nn.ConvTranspose2d(inChannels, outChannels, 2, stride=strides, padding=1)
        self.conv = convBlock(outChannels, outChannels, True, strides, 0, 0.25)

    def forward(self, X, skipFeatures):
        Y = self.convTrans(X)
        Y = torch.cat(X, skipFeatures)
        return self.conv(Y)

In [ ]:
def evaluate_accuracy(net, testIter, lossFunc, device=None):
    if isinstance(net, nn.Module):
        net.eval()
        if not device:
            device = next(iter(net.parameters())).device

    metric = d2l.Accumulator(3)

    with torch.no_grad():
        for X, y in testIter:
            X = X.to(device)
            y = y.to(device)

            yhat = net(X)

            prediction = torch.round(yhat).to(device)

            metric.add(d2l.accuracy(prediction, y) / torch.numel(y), torch.numel(y), lossFunc(yhat, y))

    return metric[0] / metric[1], metric[2] / len(testIter)

In [ ]:
def train(net: nn.Sequential, trainIter, testIter, numEpochs, startEpoch, learnRate, batchSize, device: torch.device, startDim, epochsToDouble, modelFileName, epochsToSave, cosineAnnealing=True, progressive=False, lossFunc = nn.BCEWithLogitsLoss()):
    print(f"Training on {device}")
    
    net.to(device)
    #Adaptive learning w/ learning rate scheduler, can use Adam
    #Cosine scheduler (can also use linear, not as widely used), cosine annealing w/ lr restart
    optimizer = torch.optim.Adam(net.parameters(), lr=learnRate)

    if cosineAnnealing:
        scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, 10, T_mult=1)
        #scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)

    numBatches = len(trainIter)

    imageFunc = transforms.ToPILImage()
    tensorFunc = transforms.ToTensor()

    bestValLoss = float('inf')

    currDim = startDim
    for epoch in range(startEpoch, numEpochs):
        if (epoch + 1) % epochsToDouble == 0 and progressive == 1:
            currDim *= 2
        elif (epoch + 1) % epochsToDouble == 0 and progressive == 2:
            currDim /= 2
        
        net.train()
        metric = d2l.Accumulator(2)

        for i, (X, y) in enumerate(trainIter):
            optimizer.zero_grad()
            y = y.to(device)

            if progressive > 0:
                X = F.interpolate(X, size=int(currDim))

            X = X.to(device)

            yhat = net(X).squeeze(1)

            prediction = torch.round(yhat).to(device)

            l = lossFunc(yhat, y)

            #print(f"Loss: {l.item()} Predictions: {yhat.tolist()} Labels: {y.tolist()}")

            l.backward()
            optimizer.step()

            if cosineAnnealing:
                scheduler.step(epoch + i / numBatches)

            metric.add(l, d2l.accuracy(prediction, y) / batchSize)

        if (epoch + 1) % epochsToSave == 0:
            torch.save(net.state_dict(), modelFileName + "Epoch" + str(epoch))

        validationAcc, validationLoss = evaluate_accuracy(net, testIter, lossFunc, device=device)

        if validationLoss < bestValLoss:
            bestValLoss = validationLoss
            torch.save(net.state_dict(), modelFileName + "BestLoss")

        #Also save on accuracy, test if they're the same

        print(f"Train Acc: {metric[1] / numBatches} Validation Acc: {validationAcc} Train Loss: {metric[0] / numBatches} Validation Loss: {validationLoss}")

        wandb.log({"Train Acc": metric[1] / numBatches,
                   "Validation Acc": validationAcc,
                   "Train Loss": metric[0] / numBatches,
                   "Validation Loss": validationLoss
                  })

In [ ]:
gc.collect()

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
#device = torch.device("cpu")

startEpoch = 0
networkFileName = ""
fileSaveName = "/content/drive/MyDrive/ReverseProgressiveLiverClassifier10EpochRestarts"

batchSize = 8
#Only use 0.01 with learning rate scheduler
learnRate = 0.001
epochs = 100
startDim = 256
epochsToDouble = 25
epochsToSave = 10
#0: not progressive, 1: progressive, 2: reverse progressive
progressive = 2

#lossFunc = BalancedCELoss(weight0=1, weight1=1.5)
lossFunc = FocalLoss(weight0=0.2, weight1=0.8, gamma=2)
#lossFunc = nn.BCEWithLogitsLoss()

wandb.init(project="LiverClassifier1",
           name="ReverseProgressive10EpochRestarts",
           config={
               "BatchSize":batchSize,
               "LearnRate":learnRate,
               "Epochs":epochs,
               "StartDimension":startDim,
               "EpochsToDouble":epochsToDouble
           })

#Can reduce number of channels to lessen memory usage
block1 = EncoderBlock(1, 16, 1)
block2 = EncoderBlock(16, 32, 1)
block3 = EncoderBlock(32, 64, 1)
block4 = EncoderBlock(64, 128, 1)
block5 = EncoderBlock(128, 256, 1)

net = nn.Sequential(block1, block2, block3, block4, block5, nn.AdaptiveAvgPool2d((1, 1)), nn.Flatten(), nn.Linear(256, 1), nn.Sigmoid())
#print(summary(net, (1, 256, 256)))

if networkFileName != "":
    net.load_state_dict(torch.load(networkFileName))

net = net.to(device)

print("Intialized model")

trainDataset = LITSBinaryDataset("drive/MyDrive/MachineLearningResearch/Datasets/PartialTrainDataset.hdf5")
validationDataset = LITSBinaryDataset("drive/MyDrive/MachineLearningResearch/Datasets/ValidationDataset.hdf5")
testDataset = LITSBinaryDataset("drive/MyDrive/MachineLearningResearch/Datasets/TestDataset.hdf5")

print("Dataset loaded")

trainIter = DataLoader(trainDataset, batch_size=batchSize, shuffle=True)
validationIter = DataLoader(validationDataset, batch_size=batchSize)
testIter = DataLoader(testDataset, batch_size=batchSize)

train(net, trainIter, validationIter, epochs, startEpoch, learnRate, batchSize, device, startDim, epochsToDouble, fileSaveName, epochsToSave, cosineAnnealing=False, progressive=progressive, lossFunc=lossFunc)

testAcc, testLoss = evaluate_accuracy(net, testIter, lossFunc, device=device)
print(f"10 Epoch Restarts: Test Accuracy: {testAcc} Test Loss: {testLoss}")

trainDataset.closeFile()
validationDataset.closeFile()
testDataset.closeFile()

wandb.finish()

Intialized model
Dataset loaded
Training on cuda
Train Acc: 0.410958904109589 Validation Acc: 0.24496644295302014 Train Loss: 0.06974626597884583 Validation Loss: 0.06774948174624067
Train Acc: 0.4006849315068493 Validation Acc: 0.24496644295302014 Train Loss: 0.06646110102126043 Validation Loss: 0.05990935097399511
Train Acc: 0.4623287671232877 Validation Acc: 0.7550335570469798 Train Loss: 0.06071338261643501 Validation Loss: 7.455821351001137
Train Acc: 0.6712328767123288 Validation Acc: 0.24496644295302014 Train Loss: 0.051581499102997454 Validation Loss: 0.14434785920342333
Train Acc: 0.678082191780822 Validation Acc: 0.24496644295302014 Train Loss: 0.05000291550404405 Validation Loss: 0.2285755072677459
Train Acc: 0.7106164383561644 Validation Acc: 0.7550335570469798 Train Loss: 0.04662974372710267 Validation Loss: 3.3278082044501054
Train Acc: 0.6626712328767124 Validation Acc: 0.24496644295302014 Train Loss: 0.04700312447058012 Validation Loss: 0.20911707156215256
Train Acc: 0.

Train Acc,▁▂▅▆▆▇▆▆▇▇▆▇▇▇▇▇█▇▇▇▇▇████████▆▆███████▄
Train Loss,█▇▆▅▄▃▄▄▂▃▃▃▃▂▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▄▄▂▁▁▁▁▂▁█
Validation Acc,▂██▂█▂████████▇██████▂▇██▇▂▂█▂█▁██▁▆▃███
Validation Loss,▁▅▃▁▁▁▂█▂▂▂▂▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▁▁▁▁▁▂
Train Acc,0.65411
Train Loss,0.06787
Validation Acc,0.75503
Validation Loss,1.32249
